### `KDE` Tutorial ( https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

def make_data(N, f=0.3, rseed=1):
    rand = np.random.RandomState(rseed)
    x = rand.randn(N)
    x[int(f * N):] += 5
    return x

x = make_data(20)

# -----------------------------------------------------------------------------------

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)

bins = np.linspace(-5, 10, 10)
hist1 = ax1.hist(x, bins=bins, density=True)
density, bins, patches = hist1
ax1.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
ax1.axis([-4, 9, -0.02, 0.3])

# -----------------------------------------------------------------------------------

# bins = np.linspace(-5, 10, 10)

# fig, ax = plt.subplots(1, 2, figsize=(12, 4),
#                        sharex=True, sharey=True,
#                        subplot_kw={'xlim':(-4, 9),
#                                    'ylim':(-0.02, 0.3)})
# fig.subplots_adjust(wspace=0.05)
# for i, offset in enumerate([0.0, 0.6]):
#     ax[i].hist(x, bins=bins + offset, density=True)
#     ax[i].plot(x, np.full_like(x, -0.01), '|k',
#                markeredgewidth=1)

# -----------------------------------------------------------------------------------

# fig, ax = plt.subplots()
# bins = np.arange(-3, 8)
# ax.plot(x, np.full_like(x, -0.1), '|k',
#         markeredgewidth=1)
# for count, edge in zip(*np.histogram(x, bins)):
#     for i in range(count):
#         ax.add_patch(plt.Rectangle((edge, i), 1, 1,
#                                    alpha=0.5))
# ax.set_xlim(-4, 8)
# ax.set_ylim(-0.2, 8)

# -----------------------------------------------------------------------------------

# x_d = np.linspace(-4, 8, 2000)
# density = sum((abs(xi - x_d) < 0.5) for xi in x)

# plt.fill_between(x_d, density, alpha=0.5)
# plt.plot(x, np.full_like(x, -0.1), '|k', markeredgewidth=1)

# plt.axis([-4, 8, -0.2, 8])

# -----------------------------------------------------------------------------------

from scipy.stats import norm
x_d = np.linspace(-4, 8, 1000)
density = sum(norm(xi).pdf(x_d) for xi in x)

fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)
ax2.fill_between(x_d, density, alpha=0.5)
ax2.plot(x, np.full_like(x, -0.1), '|k', markeredgewidth=1)
ax2.axis([-4, 8, -0.2, 5])

### `KDE` using `Surface Area`

In [ ]:
import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity

# -----------------------------------------------------------------------------------

def log(base, x):
    return np.log(x) / np.log(base)

# -----------------------------------------------------------------------------------

xlsx_path = r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\FuncTest_with_ipynb\clustering\{20230424_Update}_Academia_Sinica_i505\data.xlsx"
df_input_xlsx :pd.DataFrame = pd.read_excel(xlsx_path, engine = 'openpyxl')

surface_area = df_input_xlsx["Trunk surface area, SA (um2)"]
surface_area = surface_area.to_numpy()
log_sa = log(10, surface_area)

hist = plt.hist(log_sa, bins=100, density=True)
density, bins, patches = hist
widths = bins[1:] - bins[:-1]
print((density * widths).sum())


# instantiate and fit the KDE model
kde = KernelDensity(bandwidth=0.01178167723136119, kernel='gaussian')
kde.fit(log_sa[:, None])

# score_samples returns the log of the probability density
logprob = kde.score_samples(bins[:, None])

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.fill_between(bins, np.exp(logprob), alpha=0.5)
# ax.plot(bins, np.exp(logprob), label='KDE')
# ax.plot(log_sa, np.full_like(log_sa, -0.01), '|k', markeredgewidth=1)

In [ ]:
from sklearn.cluster import KMeans

# 假設已經有了一個KDE模型 kde

# 得到KDE的機率密度值
logprob = kde.score_samples(bins[:, None])
density = np.exp(logprob)

# 使用KMeans將資料分成兩群
kmeans = KMeans(n_clusters=2, random_state=0).fit(log_sa[:, None])
print(kmeans.cluster_centers_)

plt.figure(fig)
ax = plt.gca()  # 取得目前選取的圖的軸物件
for c in kmeans.cluster_centers_:
    ax.axvline(x=c, color='k', linestyle='--')

# Assign a color to each data point based on its cluster label
colors = ['r' if label == 0 else 'b' for label in kmeans.labels_]

# Scatter plot of the data colored by cluster label
ax.scatter(log_sa, np.zeros_like(log_sa), c=colors, alpha=0.5, label='KMeans clusters')

# Set the plot title and legend
ax.set_title('KDE with KMeans Clusters')
ax.legend()

# Display the plot
plt.show()

### Find best `bandwidth`

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

bandwidths = 10 ** np.linspace(-2, 2, 10000)
grid = GridSearchCV(KernelDensity(kernel='gaussian'),
                    {'bandwidth': bandwidths},
                    cv=LeaveOneOut())
grid.fit(log_sa[:, None])

print(bandwidths)
grid.best_params_

### Log Function ( any base )

In [ ]:
def log(base, x):
    return np.log(x) / np.log(base)

a = log(10, 3069901.70)
b = np.log10(3069901.70)
a-b

### Run

### Pure xlsx file ( `data.xlsx` )

In [ ]:
import sys
from pathlib import Path
sys.path.append("./../../modules/") # add path to scan customized module
from SurfaceAreaKMeansCluster import SurfaceAreaKMeansCluster

kmeans_rnd = 2022

# -----------------------------------------------------------------------------------
pure_i409_xlsx_path = Path( r"C:\Users\confocal_microscope\Desktop\WorkingDir\ZebraFish_DB\{Data}_Preprocessed\{20230424_Update}_Academia_Sinica_i505\data.xlsx" )

# -----------------------------------------------------------------------------------
# n_clusters = 3
# label_str = ["S", "M", "L"]

# -----------------------------------------------------------------------------------
n_clusters = 4
label_str = ["S", "M", "L", "XL"]

# -----------------------------------------------------------------------------------
for log_scale in [False, True]:
    for kde in [False, True]:
        SAKMeansCluster = SurfaceAreaKMeansCluster(pure_i409_xlsx_path, n_clusters, label_str, kmeans_rnd,
                                                    log_base=10, cluster_with_log_scale=log_scale, with_kde=kde)
        print("="*80, "\n"); print(SAKMeansCluster); SAKMeansCluster.plot_and_save_xlsx()

### Run all of old xlsx ( i409, i505 )( `{'n'CLS_BY_SurfStDev}_data.xlsx` )

In [ ]:
import sys
from pathlib import Path
sys.path.append("./../../modules/") # add path to scan customized module
from SurfaceAreaKMeansCluster import SurfaceAreaKMeansCluster

kmeans_rnd = 2022

# -----------------------------------------------------------------------------------

old_i409_3c_path = Path( r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\FuncTest_with_ipynb\clustering\{20230305_NEW_STRUCT}_Academia_Sinica_i409\{3CLS_BY_SurfStDev}_data.xlsx" )
n_clusters = 3
label_str = ["S", "M", "L"]
i409_3c_sheet_name = ["0.5_STDEV"]

for old_classdiv_sheet_name in i409_3c_sheet_name:
    for log_scale in [False, True]:
        for kde in [False, True]:
            SAKMeansCluster = SurfaceAreaKMeansCluster(old_i409_3c_path, n_clusters, label_str, kmeans_rnd,
                                                       log_base=10, cluster_with_log_scale=log_scale, with_kde=kde,
                                                       old_classdiv_sheet_name=old_classdiv_sheet_name)
            print("="*80, "\n"); print(SAKMeansCluster); SAKMeansCluster.plot_and_save_xlsx()


# -----------------------------------------------------------------------------------

old_i505_3c_path = Path( r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\FuncTest_with_ipynb\clustering\{20230424_Update}_Academia_Sinica_i505\{3CLS_BY_SurfStDev}_data.xlsx" )
n_clusters = 3
label_str = ["S", "M", "L"]
i505_3c_sheet_name = ["0.5_STDEV", "0.75_STDEV", "1.0_STDEV"]

for old_classdiv_sheet_name in i505_3c_sheet_name:
    for log_scale in [False, True]:
        for kde in [False, True]:
            SAKMeansCluster = SurfaceAreaKMeansCluster(old_i505_3c_path, n_clusters, label_str, kmeans_rnd,
                                                       log_base=10, cluster_with_log_scale=log_scale, with_kde=kde,
                                                       old_classdiv_sheet_name=old_classdiv_sheet_name)
            print("="*80, "\n"); print(SAKMeansCluster); SAKMeansCluster.plot_and_save_xlsx()


# -----------------------------------------------------------------------------------

old_i505_4c_path = Path( r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\FuncTest_with_ipynb\clustering\{20230424_Update}_Academia_Sinica_i505\{4CLS_BY_SurfStDev}_data.xlsx" )
n_clusters = 4
label_str = ["S", "M", "L", "XL"]
i505_4c_sheet_name = ["0.5_STDEV", "0.75_STDEV", "1.0_STDEV"]

for old_classdiv_sheet_name in i505_4c_sheet_name:
    for log_scale in [False, True]:
        for kde in [False, True]:
            SAKMeansCluster = SurfaceAreaKMeansCluster(old_i505_4c_path, n_clusters, label_str, kmeans_rnd,
                                                       log_base=10, cluster_with_log_scale=log_scale, with_kde=kde,
                                                       old_classdiv_sheet_name=old_classdiv_sheet_name)
            print("="*80, "\n"); print(SAKMeansCluster); SAKMeansCluster.plot_and_save_xlsx()